In [28]:
import warnings
warnings.filterwarnings('ignore')

In [29]:
import pandas as pd
from neuralprophet import NeuralProphet
import plotly.graph_objects as go
import pickle
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator

In [30]:
df = pd.read_csv('./dataset/sensors_202205211506.csv', parse_dates=['created_at'], index_col='created_at')


cols = ['temperature']
df = df[cols].astype(float)


In [31]:
freq = '1min'

In [32]:
df = df.resample(freq).mean().interpolate()

In [ ]:
df.info()

In [34]:
df = df.reset_index()

In [ ]:
temperature_data = df['temperature'].values
temperature_data = temperature_data.reshape((-1,1))

split_percent = 0.80
split = int(split_percent*len(temperature_data))

temperature_train = temperature_data[:split]
temperature_test = temperature_data[split:]

date_train = df['created_at'][:split]
date_test = df['created_at'][split:]

print(len(temperature_train))
print(len(temperature_test))

In [36]:
look_back = 15

train_generator = TimeseriesGenerator(temperature_train, temperature_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(temperature_test, temperature_test, length=look_back, batch_size=1)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense,SimpleRNN

model = Sequential()
# model.add(
#     LSTM(10,
#         activation='relu',
#         input_shape=(look_back,1))
# )
# model.add(Dense(1))
model.add(SimpleRNN(units=32, input_shape=(look_back,1), activation="relu"))
model.add(Dense(8, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 25
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)


In [ ]:
prediction = model.predict_generator(test_generator)

temperature_train = temperature_train.reshape((-1))
temperature_test = temperature_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = temperature_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = temperature_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Temperature forecast",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "temp"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

In [39]:
temperature_data = temperature_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = temperature_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['created_at'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1,freq=freq).tolist()
    return prediction_dates

num_prediction = 60*24
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

In [ ]:
trace1 = go.Scatter(
    x = df['created_at'].values,
    y = df['temperature'].values,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode = 'lines',
    name = 'Prediction'
)

layout = go.Layout(
    title = "Temperature forecast",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "temp"}
)
fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()